In [1]:
from scads import Scads, add_conceptnet, add_datasets
from modules.module import BaseModule
from taglet_executer import TagletExecuter
from task import Task
from JPL_interface import JPL
import os
import random
import pandas as pd
from pathlib import Path
from modules.module import *
%load_ext autoreload
%autoreload 2

In [2]:
#####***** connect to JPL API and create a session ********#####

JPL_API = JPL()
tasks = JPL_API.get_available_tasks()
task_name = tasks[1]    # Problem_test_image_classification

task_metadata = JPL_API.get_task_metadata(task_name)
JPL_API.create_session(task_name)


#####***** creat task based on the meta data from curren session ********#####
current_task = Task(task_metadata)
session_status = JPL_API.get_session_status()
current_dataset = session_status['current_dataset']
current_task.classes = current_dataset['classes']
current_task.number_of_channels = current_dataset['number_of_channels']
###download data to local system in the path given by data_url(/datasets/lwll_datasets/mnist/mnist_sample/train)

current_task.unlabeled_image_path = '.'+ current_dataset['data_url']
current_task.test_image_path = os.path.abspath(os.getcwd())+'/datasets/lwll_datasets/mnist/mnist_sample/test/' # should be updated later
current_task.labeled_images = JPL_API.get_seed_labels()

In [3]:
#####***** get available budget and run active learning module ********#####

available_budget = session_status['budget_left_until_checkpoint']
active_learning = RandomActiveLearning(current_task, available_budget)
active_learning.find_candidates()
examples = active_learning.candidates

query = {'example_ids':examples}

labeled_images = JPL_API.request_label(query)
current_task.labeled_images = labeled_images
print(len(labeled_images))


./datasets/lwll_datasets/mnist/mnist_sample/train
3000


In [15]:
#####***** create a transfer learning module ********#####
MNIST_module = TransferModule(task=current_task)

number of training data: 2100
number of validation data: 450
number of test data: 450


In [16]:
#####***** train taglets assocated with the module ********#####
MNIST_module.train_taglets()

epoch: 0
train loss: 0.0022
train acc: 0.1905%
validation loss: 0.0153
validation acc: 2.0000%
Deep copying new best model. (validation of 0.0200%, over 0.0000%)
Epoch 1 result: 
Average trainin loss: 0.0022
Average training accuracy: 0.1905%
Average validation loss: 0.0153
Average validation accuracy: 2.0000%
test loss: 0.0153
test acc: 2.6667%


In [17]:
#####***** create taglet executer model ********#####
taglets = MNIST_module.get_taglets()
taglet_executer = TagletExecuter(taglets)

In [18]:
#####***** perform inference on ALL unlabeled images using taglet_executer and return label matrix ********#####
label_matrix = taglet_executer.execute(current_task.unlabeled_images)

In [ ]:
#####***** labelmodel on label_matrix and train end model ********#####
# soft_labels = LabelModel.annotate(label_matrix)
# end_model = end_model(...)
# [test_predictions] = end_model.prediction(end_model, task.evaluation_image_path)

### random labels just for the sake of producing something...
test_imgs = [f.name for f in Path(current_task.test_image_path).iterdir() if f.is_file()]
rand_lbls = [str(random.randint(0, 10)) for _ in range(len(test_imgs))]
df = pd.DataFrame({'id': test_imgs, 'label': rand_lbls})
predictions = df.to_dict()


In [ ]:
#####************* submit predictions ***********#####

JPL_API.submit_prediction(predictions)

session_status = JPL_API.get_session_status()
print(session_status['checkpoint_scores'])
print('we are currently in '+ session_status['pair_stage']+ ' phase!')